Train Data qa_generation

In [6]:
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    RobertaModel, RobertaPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import load_from_disk

train_dataset = load_from_disk('data/train_dataset/train/')
valid_dataset = load_from_disk('data/train_dataset/validation/')

In [11]:
train_dataset[0]

{'title': '미국 상원',
 'answers': {'answer_start': [235], 'text': ['하원']},
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 '__index_level_0__': 42,
 'document_id': 1829

In [14]:
len(train_dataset['context'])

3952

In [15]:
from pororo import Pororo
ner = Pororo(task="ner", lang="ko")

[Korean Sentence Splitter]: Initializing Kss...




In [23]:
a = ner(train_dataset[0]['context'])

In [32]:
ner_pairs = tuple(set(a))
ner_list = []

# ner tags
# PS": "PERSON","LC": "LOCATION","OG": "ORGANIZATION","AF": "ARTIFACT",
# "DT": "DATE","TI": "TIME","CV": "CIVILIZATION","AM": "ANIMAL",
# "PT": "PLANT","QT": "QUANTITY","FD": "STUDY_FIELD",
# "TR": "THEORY","EV": "EVENT","MT": "MATERIAL","TM": "TERM",

for k, v in a:
    if v != 'O':
        ner_list.append((k, v))
list(set(ner_list))

[('국민', 'CIVILIZATION'),
 ('캘리포니아주', 'LOCATION'),
 ('2년마다', 'DATE'),
 ('수반', 'CIVILIZATION'),
 ('내각책임제', 'CIVILIZATION'),
 ('양원제', 'CIVILIZATION'),
 ('상원', 'ORGANIZATION'),
 ('의장', 'OCCUPATION'),
 ('50개주', 'QUANTITY'),
 ('2년', 'DATE'),
 ('2명', 'QUANTITY'),
 ('의원', 'OCCUPATION'),
 ('국회', 'ORGANIZATION'),
 ('미국', 'COUNTRY'),
 ('공무원', 'OCCUPATION'),
 ('2017-02-05', 'DATE'),
 ('100명', 'QUANTITY'),
 ('연방 행정부', 'ORGANIZATION'),
 ('상의원', 'CIVILIZATION'),
 ('하원', 'ORGANIZATION'),
 ('퍼블릭 옵션', 'CIVILIZATION'),
 ('1/3씩', 'QUANTITY'),
 ('관료', 'OCCUPATION'),
 ('대통령', 'CIVILIZATION'),
 ('정부', 'ORGANIZATION'),
 ('단', 'CIVILIZATION'),
 ('부통령', 'OCCUPATION'),
 ('일리노이주', 'LOCATION'),
 ('6년', 'DATE'),
 ('원제', 'CIVILIZATION')]

In [ ]:
# pororo ner, gptgeneration으로 문장생성후 pororo semantic_textual_similarity 로 q,p 유사도 낮은거위주로 넣기
# pororo sentence embedding으로 dense retrieval 해보기

# pororo paraphrase, question generation # pororo qg는 korquad기반임.. ㅜ 
# pg는 multilingual internal dataset seq2seq bart

# pororo summarization으로 paraphrase만 요약해서 어그멘테이션 해보기

In [2]:
import pandas as pd

wiki = pd.read_json('data/wikipedia_documents.json', orient='index')
wiki

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,NaN,나라 목록,NaN,NaN,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,NaN,나라 목록,NaN,NaN,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,NaN,백남준,NaN,NaN,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,NaN,아오조라 문고,NaN,NaN,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,NaN,아오조라 문고,NaN,NaN,4
...,...,...,...,...,...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,NaN,오키나와 현립 박물관·미술관,NaN,NaN,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,NaN,오키나와 현립 박물관·미술관,NaN,NaN,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,NaN,오키나와 현립 박물관·미술관,NaN,NaN,60610
60611,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,NaN,단결권 및 단체교섭권 협약,NaN,NaN,60611


In [4]:
len(wiki['text'].unique())

56737

In [5]:
!git clone https://github.com/codertimo/KorQuAD-Question-Generation qa_generation

Cloning into 'qa_generation'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 125 (delta 54), reused 105 (delta 39), pack-reused 0
Receiving objects: 100% (125/125), 926.44 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (54/54), done.


하겸님의 question <-> title 추가

In [ ]:
texts = []
titles = []

for i in range(len(wiki_df)):
    wiki_context = wiki_df['text'][i]
    wiki_title = wiki_df['title'][i]

    if wiki_title in wiki_context:
        texts.append(wiki_context)
        titles.append(wiki_title)

wiki_qa_df = pd.DataFrame(data={'text':texts,'title':titles})
wiki_qa_df.head()

In [ ]:
def load_wiki_dataset(dataset_path):
    wiki_data_frame = pd.read_csv(dataset_path)
    examples = []
    for idx in range(len(wiki_data_frame)):
        text = wiki_data_frame["text"][idx]
        title = wiki_data_frame["title"][idx]

        example = QAExample(text, title)
        examples.append(example)
    return examples

In [ ]:
import sys
sys.path.append("..")
import random
from argparse import ArgumentParser

import torch
from tokenizers import SentencePieceBPETokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import GPT2LMHeadModel, AutoModelWithLMHead, AutoTokenizer

from korquad_qg.config import QGConfig
from korquad_qg.dataset import (MAX_QUESTION_SPACE, MIN_QUESTION_SPACE, QGDecodingDataset, load_wiki_dataset)

model = AutoModelWithLMHead.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
# model.load_state_dict(torch.load('/opt/ml/KorQuAD-Question-Generation/model/QG_kogpt2.pth', map_location="cpu"))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer = SentencePieceBPETokenizer.from_file(
    vocab_filename="../tokenizer/vocab.json", merges_filename="../tokenizer/merges.txt", add_prefix_space=False
)

examples = load_wiki_dataset('/opt/ml/data/wiki_text_title.csv')
random.shuffle(examples)
dataset = QGDecodingDataset(examples, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=1)

model = model.to(device)
model.eval()

generated_results = []

for i, batch in tqdm(enumerate(dataloader), desc="generate", total=len(dataloader)):
    input_ids, attention_mask = (v.to(device) for v in batch)
    origin_seq_len = input_ids.size(-1)

    decoded_sequences = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=origin_seq_len + MAX_QUESTION_SPACE,
        min_length=origin_seq_len + MIN_QUESTION_SPACE,
        pad_token_id=0,
        bos_token_id=1,
        eos_token_id=2,
        num_beams=5,
        repetition_penalty=1.3,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
    )

    for decoded_tokens in decoded_sequences.tolist():
        decoded_question_text = tokenizer.decode(decoded_tokens[origin_seq_len:])
        decoded_question_text = decoded_question_text.split("</s>")[0].replace("<s>", "")
        generated_results.append(
            (examples[i].context, examples[i].answer, examples[i].question, decoded_question_text)
        )

with open('test.tsv', "w") as f:
    for context, answer, question, generated_question in generated_results:
        f.write(f"{generated_question}\t{answer}\t{context}\n")